## Reaching attempt


In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import trange
from gymnasium.utils.env_checker import check_env
from IPython.display import Video
from enum import Enum, auto

from flygym.mujoco import Parameters
from flygym.mujoco.arena import FlatTerrain
from flygym.mujoco.examples.obstacle_arena import ObstacleOdorArena
from flygym.mujoco.examples.turning_controller import HybridTurningNMF
from flygym.mujoco import Parameters, NeuroMechFly

from flygym.mujoco.examples.common import PreprogrammedSteps
import numpy as np

import matplotlib.pyplot as plt

## Setting up Basic Sim

In [33]:
import flygym.mujoco
from tqdm import trange

# We start by creating a simple arena
flat_terrain_arena = FlatTerrain()

# Then, we add visual and olfactory features on top of it
arena = ObstacleOdorArena(
    terrain=flat_terrain_arena,
    obstacle_positions=np.array([(7.5, 0), (12.5, 5), (17.5, -5)]),
    marker_size=0.5,
    obstacle_colors=[(0.14, 0.14, 0.2, 1), (0.2, 0.8, 0.2, 1), (0.2, 0.2, 0.8, 1)],
    user_camera_settings=((13, -18, 9), (np.deg2rad(65), 0, 0), 45),
)

contact_sensor_placements = [
    f"{leg}{segment}"
    for leg in ["LF", "LM", "LH", "RF", "RM", "RH"]
    for segment in ["Tibia", "Tarsus1", "Tarsus2", "Tarsus3", "Tarsus4", "Tarsus5"]
]

run_time = 1
sim_params = flygym.mujoco.Parameters(
    timestep=1e-4, 
    render_mode="saved", 
    render_playspeed=0.1, 
    draw_contacts=False,
    render_camera="Animat/camera_top"
)

nmf = HybridTurningNMF(
    sim_params=sim_params,
    init_pose="stretch",
    spawn_pos=(13, -5, 0.2),
    spawn_orientation=(0, 0, np.pi / 2 + np.deg2rad(70)),
    contact_sensor_placements=contact_sensor_placements,
    arena=arena,
)

## Things for reaching..

In [40]:
def create_standing_action():
    preprogrammed_steps = PreprogrammedSteps()
    swing_periods = preprogrammed_steps.swing_period
    legs = preprogrammed_steps.legs

    standing_action = []
    for leg in legs:
        if leg.endswith("M"):
            standing_action.extend(
                preprogrammed_steps.get_joint_angles(leg, swing_periods[leg][1])
            )
        else:
            standing_action.extend(
                preprogrammed_steps.get_joint_angles(leg, 0.0)
            )

    standing_action = {"joints": standing_action, "adhesion": np.zeros(len(legs))}
    return standing_action

def make_stand(stand_action):
    for i in range(int(0.2 // nmf.timestep)):
        nmf.step(stand_action, True)

def create_lunge_base(nmf):
    preprogrammed_steps = PreprogrammedSteps()
    swing_periods = preprogrammed_steps.swing_period
    legs = preprogrammed_steps.legs

    run_time = 0.3
    target_num_steps = int(run_time // nmf.timestep)
    
    foreleg_ids = np.zeros(target_num_steps)
    middle_stance_ids = np.linspace(swing_periods["RM"][1], 2 * np.pi, target_num_steps)
    hind_swing_ids = np.linspace(0.0, swing_periods["RH"][1], target_num_steps)

    adhesion_action = np.array([0.0 if leg.endswith("F") else 1.0 for leg in legs])

    all_joint_angles = []

    for i in range(target_num_steps):
        joint_angles = []
        for leg in legs:
            if leg.endswith("F"):
                joint_angles.extend(
                    preprogrammed_steps.get_joint_angles(leg, foreleg_ids[i])
                )
            elif leg.endswith("M"):
                joint_angles.extend(
                    preprogrammed_steps.get_joint_angles(leg, middle_stance_ids[i])
                )
            else:
                joint_angles.extend(
                    preprogrammed_steps.get_joint_angles(leg, hind_swing_ids[i])
                )
        all_joint_angles.append(joint_angles.copy())
        lunge_base_action = {"joints": np.array(joint_angles), "adhesion": adhesion_action}
        return lunge_base_action

def create_extend_action(nmf):
    preprogrammed_steps = PreprogrammedSteps()
    swing_periods = preprogrammed_steps.swing_period
    legs = preprogrammed_steps.legs

    run_time = 0.15
    target_num_steps = int(run_time // nmf.timestep)

    foreleg_ids = np.zeros(target_num_steps)
    foreleg_stance_ids = np.linspace(swing_periods["RF"][1], 2 * np.pi, target_num_steps)
    middle_stance_ids = np.linspace(swing_periods["RM"][1], 2 * np.pi, target_num_steps)
    hind_swing_ids = np.linspace(0.0, swing_periods["RH"][1], target_num_steps)

    R_midleg_start = preprogrammed_steps.get_joint_angles("RM", swing_periods["RM"][1])
    R_midleg_stretch = np.linspace(
        np.zeros(len(R_midleg_start)), -R_midleg_start, target_num_steps
    )

    L_midleg_start = preprogrammed_steps.get_joint_angles("LM", swing_periods["LM"][1])
    L_midleg_stretch = np.linspace(
        np.zeros(len(L_midleg_start)), -L_midleg_start, target_num_steps
    )

    R_frontleg_start = preprogrammed_steps.get_joint_angles("RF", swing_periods["RF"][1])
    R_frontleg_stretch = np.linspace(
        np.zeros(len(R_frontleg_start)), -R_midleg_start, target_num_steps
    )

    adhesion_action = np.array([0.0 if leg.endswith("F") else 1.0 for leg in legs])

    all_joint_angles = []
    extend_action_array = []

    for i in range(target_num_steps):
        joint_angles = []
        for leg in legs:
            if leg.endswith("F"):
                frontleg_joint_angles = preprogrammed_steps.get_joint_angles(
                    leg, foreleg_stance_ids[i]
                )
                if leg.startswith("R"):
                    frontleg_joint_angles += R_frontleg_stretch[i]/2
                joint_angles.extend(frontleg_joint_angles)
            elif leg.endswith("M"):
                midleg_joint_angles = preprogrammed_steps.get_joint_angles(
                    leg, middle_stance_ids[i]
                )
                if leg.startswith("R"):
                    midleg_joint_angles += R_midleg_stretch[i]/4
                elif leg.startswith("L"):
                    midleg_joint_angles += L_midleg_stretch[i]/4
                joint_angles.extend(midleg_joint_angles)
            else:
                joint_angles.extend(
                    preprogrammed_steps.get_joint_angles(leg, hind_swing_ids[i])
                )

        all_joint_angles.append(joint_angles.copy())
        extend_action = {"joints": np.array(joint_angles), "adhesion": adhesion_action}
        extend_action_array.append(extend_action)
    
    return extend_action_array

def create_extend_one_action(nmf):
    preprogrammed_steps = PreprogrammedSteps()
    swing_periods = preprogrammed_steps.swing_period
    legs = preprogrammed_steps.legs

    run_time = 0.3
    target_num_steps = int(run_time // nmf.timestep)

    foreleg_ids = np.zeros(target_num_steps)
    foreleg_stance_ids = np.linspace(swing_periods["RF"][1], 2 * np.pi, target_num_steps)
    middle_stance_ids = np.linspace(swing_periods["RM"][1], 2 * np.pi, target_num_steps)
    hind_swing_ids = np.linspace(0.0, swing_periods["RH"][1], target_num_steps)

    R_midleg_start = preprogrammed_steps.get_joint_angles("RM", swing_periods["RM"][1])
    R_midleg_stretch = np.linspace(
        np.zeros(len(R_midleg_start)), -R_midleg_start, target_num_steps
    )

    L_midleg_start = preprogrammed_steps.get_joint_angles("LM", swing_periods["LM"][1])
    L_midleg_stretch = np.linspace(
        np.zeros(len(L_midleg_start)), -L_midleg_start, target_num_steps
    )

    adhesion_action = np.array([0.0 if leg.endswith("F") else 1.0 for leg in legs])

    all_joint_angles = []
    extend_action_array = []

    for i in range(target_num_steps):
        joint_angles = []
        for leg in legs:
            if leg == "RM" :
                joint_angles.extend(
                    preprogrammed_steps.get_joint_angles(leg, foreleg_ids[i])
                )
            elif leg.endswith("M"):
                midleg_joint_angles = preprogrammed_steps.get_joint_angles(
                    leg, middle_stance_ids[i]
                )
                # if leg.startswith("R"):
                #     midleg_joint_angles += R_midleg_stretch[i]
                # elif leg.startswith("L"):
                #     midleg_joint_angles += L_midleg_stretch[i]
                joint_angles.extend(midleg_joint_angles)
            else:
                joint_angles.extend(
                    preprogrammed_steps.get_joint_angles(leg, hind_swing_ids[i])
                )

        all_joint_angles.append(joint_angles.copy())
        extend_action = {"joints": np.array(joint_angles), "adhesion": adhesion_action}
        extend_action_array.append(extend_action)
    
    return extend_action_array

    
    
        


## Run simple test sim for lunge base

In [41]:
from enum import Enum, auto

# random state seed for reproducibility
seed = 1

decision_interval = 0.2
run_time = 1
num_decision_steps = int(run_time / decision_interval)
physics_steps_per_decision_step = int(decision_interval / sim_params.timestep)

standing_action = create_standing_action()
lunge_base_action = create_lunge_base(nmf)
extend_action_array = create_extend_action(nmf)

obs_hist = []
odor_history = []
obs, _ = nmf.reset(seed)
bias = np.array([0,0])
for i in trange(int(run_time / nmf.sim_params.timestep)):
    curr_time = i * nmf.sim_params.timestep
    left_sense = np.array(obs["contact_forces"][:5, 0:2])
    right_sense = np.array(obs["contact_forces"][18:23, 0:2])

    if curr_time < 0.15:
        control_signal = np.array([1, 1])
        obs, reward, terminated, truncated, info = nmf.step(control_signal, False)
    elif curr_time > 0.2 and curr_time < 0.4:
        nmf.step(standing_action, True)
    else:
        if len(extend_action_array) >= 1:
            extend_action = extend_action_array[0]
            del extend_action_array[0]
            nmf.step(extend_action, True)
        else:
            control_signal = np.array([1, 1])
            obs, reward, terminated, truncated, info = nmf.step(control_signal, False)
    nmf.render()

nmf.save_video("./outputs/lunging_base.mp4")
Video("./outputs/lunging_base.mp4")

100%|██████████| 10000/10000 [01:05<00:00, 153.12it/s]
